### ECE-GY 6123
### Nikita Makarov, Fall 2024

### Homework 2

Problem 1a:

In [ ]:
from PIL import Image
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

# Create conv2 function
def conv2(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    '''
    Compute and return the same-padded 2D convolution of two matrices A and B.
    
    Parameters:
    A: 2D numpy array of the first matrix
    B: 2D numpy array of the second matrix

    Returns:
    C: 2D numpy array of the same-padded 2D convolution matrix
    '''